# Topic Modeling using LDA

In [1]:
# # To be run only once
# if 0 == 1:
#     !pip install gensim
#     !pip install PyLDAvis
#     !pip install spacy
#     !python -m spacy download en_core_web_sm

In [2]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt

import re
import spacy
import tqdm

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import en_core_web_sm

from pprint import pprint

import pyLDAvis.gensim
import pickle 
import pyLDAvis

import time
from collections import Counter
import random

In [3]:
random.seed(3)

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asaid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Preprocessing data

In [6]:
start = time.time()

In [7]:
# one_drive_path = "C:/Users/cfowle/The Estée Lauder Companies Inc/TeamAnis - General/"
one_drive_path = "C:/Users/asaid/The Estée Lauder Companies Inc/TeamAnis - General/"

In [8]:
# reviews = pd.read_pickle('reviews.pickle')

# cols = ['type', 'onlinepost_id', 'source_product_identifier', 'onlinestatement_id',
#        'date', 'title', 'description', 'geography', 'channel', 'product_id',
#        'rating', 'sentiment']
# reviews = reviews[cols]

# reviews.loc[reviews['type']=='Cosmetics','type']='temps'
# reviews.loc[reviews['type']=='Skincare','type']='Cosmetics'
# reviews.loc[reviews['type']=='temps','type']='Skincare'

# reviews = reviews[reviews['geography']=='USA']
# reviews.drop(columns=['geography'], inplace=True)

# reviews.to_pickle('reviews_filtered.pickle')

In [9]:
# reviews = pd.read_pickle('reviews_filtered.pickle')

In [10]:
# # Creating date columns in the right dtype and dropping the day of the month: 2019-02-24 => 2019-02-01
# reviews.loc[:,'clean_date'] = pd.to_datetime(reviews['date'], errors='coerce')
# if reviews['clean_date'].isna().sum() > 0:
#     print('{} rows have been dropped because the date format is wrong.'.format(reviews['clean_date'].isna().sum()))
#     display(reviews.loc[reviews['clean_date'].isna(), 'date'])
#     reviews = reviews.dropna(subset='date')
# reviews['date'] = reviews['clean_date']  
# reviews = reviews.drop('clean_date', axis=1)
# reviews['date'] = reviews['date'].dt.to_period('m')

# # Checking for missing data (NA => -1)
# if reviews['rating'].isna().sum()>0:
#     print('{} rows are missing ratings'.format(reviews['rating'].isna().sum()))
#     reviews.loc[:,'rating'] = reviews['rating'].fillna(-1).astype(int)

# if reviews['sentiment'].isna().sum()>0:
#     print('{} rows are missing sentiments'.format(reviews['sentiment'].isna().sum()))
#     reviews.loc[:,'sentiment'] = reviews['sentiment'].fillna(-1).astype(int)
    
# # Transforming rating and sentiment to dummy variables (one-hot encoding)
# reviews.loc[:,'sentiment'] = reviews['sentiment'].str.lower()
# reviews.loc[:,'rating'] = reviews['rating'].astype(int)
# reviews = pd.concat([reviews, pd.get_dummies(data=reviews[['rating','sentiment']], columns=['rating','sentiment'], dtype=int)], axis=1)

# # Readding NAs data to ratings
# reviews.loc[reviews['rating']==-1,'rating'] = np.nan
# reviews.loc[reviews['sentiment']==-1,'sentiment'] = np.nan

# # Transforming sentiment to integer data (positive:1; netural:0, negative:-1)
# reviews.loc[:,'sentiment'] = reviews['sentiment_positive'] - reviews['sentiment_negative']

# # Aggregating RR data by OnlinePost_ID

# # Creating a column to count the number of statements by review once aggreagtion happens
# reviews['nb_statements'] = reviews['sentiment']

# reviews = reviews.groupby(['type',
#                            'channel',
#                            'source_product_identifier',  
#                            'date',
#                            'onlinepost_id']).agg({
#     'description': lambda x:'. '.join(list(x)),
#     'nb_statements':'count',
#     'rating':'first',
#     'rating_1':'first',
#     'rating_2':'first',
#     'rating_3':'first',
#     'rating_4':'first',
#     'rating_5':'first',
#     'sentiment_negative':'sum',
#     'sentiment_neutral':'sum',
#     'sentiment_positive':'sum',
#     'sentiment':'mean'
# }).reset_index()

# # Normalize the one hot sentiment encoding counts (sentiment_negative, sentiment_neutral, sentiment_positive) by the nb_statement.
# reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']] = reviews[['sentiment_negative', 'sentiment_neutral', 'sentiment_positive']].div(reviews['nb_statements'], axis=0)

# reviews.to_pickle('reviews_wrangled.pickle')

In [11]:
reviews = pd.read_pickle('reviews_wrangled.pickle')

In [12]:
# Adding product related words to the stop words
stop_words = stopwords.words('english')
stop_words.extend(['from'])

brands = pd.read_csv('elc_brands.csv', encoding='ISO-8859-1')
catalogue = pd.read_csv('elc_catalogue.csv', encoding='ISO-8859-1')

stop_words.extend(list(set(str(w).lower() for w in ' '.join(brands['ELC_Brand'].unique().tolist() +
                                          catalogue['Major_Category'].unique().tolist() +
                                          catalogue['Application'].unique().tolist() +
                                          catalogue['Category'].unique().tolist() +
                                          catalogue['SubCategory'].unique().tolist() 
                                         ).replace('/', ' ').split())))

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
stop_words_nlp = nlp(' '.join([' '.join(gensim.utils.simple_preprocess(str(word), deacc=True)) for word in stop_words]))
stop_words = [token.lemma_ for token in stop_words_nlp]


# Cache stop_words into hash
stop_words = Counter(stop_words)


def preprocess(sentences, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    for sentence in sentences:
        doc = nlp(' '.join([token for token in gensim.utils.simple_preprocess(str(sentence), deacc=True)]) )
        yield([token.lemma_ for token in doc if (token.pos_ in allowed_postags) and (not (token.lemma_ in stop_words))])

In [13]:
'lipstick' in stop_words

True

In [14]:
reviews['tokens'] = list(tqdm.tqdm(preprocess(reviews['description'].values.tolist(), stop_words), position=0, leave=True, total=len(reviews)))

100%|█████████████████████████████████████████████████████████████████████| 4636608/4636608 [4:21:43<00:00, 295.26it/s]


In [15]:
reviews.to_pickle('reviews_wrangled_w_tokens.pickle')

In [16]:
reviews = pd.read_pickle('reviews_wrangled_w_tokens.pickle')

In [17]:
rp = pd.read_csv('reviews_products.csv', low_memory=False)
products = rp[['type', 'channel', 'source_product_identifier', 'product', 'brand_abbrev', 'elc_brand',
       'brand_score', 'item_description', 'product_score', 'itemid_4',
       'major_category_id', 'major_category', 'application_id', 'application',
       'category_id', 'category', 'sub_category_id', 'sub_category']].drop_duplicates()
products = products.dropna(subset=['elc_brand'], axis=0)

In [18]:
reviews  = reviews.merge(products, left_on=['type', 'channel', 'source_product_identifier'], right_on=['type', 'channel', 'source_product_identifier'], how='inner')

In [19]:
reviews.to_pickle('reviews_products.pickle')

In [6]:
reviews = pd.read_pickle('reviews_products.pickle')

In [7]:
reviews

,type,channel,source_product_identifier,date,onlinepost_id,description,nb_statements,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,tokens,product,brand_abbrev,elc_brand,brand_score,item_description,product_score,itemid_4,major_category_id,major_category,application_id,application,category_id,category,sub_category_id,sub_category
0,Skincare,Amazon USA,B00015GYC2,2015-03,OnlinePost_20200301_24654a95-b252-4dee-8dfa-77...,This was very disappointing.. When I received ...,4,1.0,1,0,0,0,0,0.5,0.500000,0.000000,-0.500000,"[disappointing, receive, cleansing, use, night...",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
1,Skincare,Amazon USA,B00015GYC2,2017-01,OnlinePost_20200301_84729e90-665d-4108-8ca5-0b...,I have tried many face washes- this is the bes...,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[try, many, good, get, dirt, soft]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
2,Skincare,Amazon USA,B00015GYC2,2017-12,OnlinePost_20200301_9c216502-fba9-48a8-95cb-3b...,Fast service.. could find it from my usual ven...,3,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[fast, could, find, usual, vendor, one, fast, ...",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
3,Skincare,Amazon USA,B00015GYC2,2018-01,OnlinePost_20200301_01d7a936-32dd-454d-b4ef-ab...,Five Stars.. great great product.,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[star, great, great]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
4,Skincare,Amazon USA,B00015GYC2,2018-07,OnlinePost_20200301_53a09274-200c-47dc-9cca-52...,Order from La Mer only.. Can't go wrong with L...,3,4.0,0,0,0,1,0,0.0,0.666667,0.333333,0.333333,"[order, go, wrong, order, site]",La Mer Cleansing Gel 6.7 oz / 200 ml,CM,La Mer,1.0,THE CLEANSING GEL,0.731931,51T0,520.0,Skincare,25,Face,34,Cleansers (Incl Soap,Z5,All Cleansers (Incl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388052,Cosmetics,Ulta,xlsImpprod6490080,2019-06,OnlinePost_20191027_184332048,Maybe I got an old one but this thing was awfu...,6,1.0,1,0,0,0,0,0.5,0.166667,0.333333,-0.166667,"[maybe, get, old, thing, awful, bad, batch, pa...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388053,Cosmetics,Ulta,xlsImpprod6490080,2019-06,OnlinePost_20191027_185330102,Love most Smashbox products and used the origi...,5,1.0,1,0,0,0,0,0.2,0.400000,0.400000,0.200000,"[love, use, original, year, disappoint, disapp...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388054,Cosmetics,Ulta,xlsImpprod6490080,2019-07,OnlinePost_20191027_184010998,The Best Primer.. this is literally the best p...,3,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[good, literally, good, ever, use, use, else, ...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime
388055,Cosmetics,Ulta,xlsImpprod6490080,2019-07,OnlinePost_20191027_184176048,I didn't wear it for 24 hours but my eyeshadow...,2,5.0,0,0,0,0,1,0.0,0.000000,1.000000,1.000000,"[wear, hour, crease, stay, entire, time, exact...",Photo Finish 24-Hour Shadow Primer,SX,Smashbox,1.0,PHOTO FNSH 24 HR SHADOW PRIMER,0.896071,C1JJ,510.0,Makeup,15,Eyes,2A,Eye Makeup Primer,MB,All Eye Makeup Prime


## Using class

In [1]:
class TopicModeller():
    def __init__(self, df, column, name):
        self.df = df.reset_index(drop=True)
        self.column = column
        self.docs = df[column].values.tolist()
        self.name = name
        
    def compute_coherence_values(self, corpus, id2word, text, k, a, b):

        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=k, 
                                               random_state=3,
                                               chunksize=100,
                                               passes=10,
                                               alpha=a,
                                               eta=b)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')

        return coherence_model_lda.get_coherence()
    
    def train_valid_lda(self,
                        nb_samples: int = 100000,
                        topics_range: list = [3,5,7,9],
                        alpha: list = [0.1, 0.01, 'symmetric','asymmetric'],
                        beta: list = [0.1, 'symmetric']                        
                        ):
        
        nb_samples = min(nb_samples, len(self.docs))
        
        # Training set
        train = random.sample(self.docs, nb_samples)
        
        # Create Dictionary
        self.id2word = corpora.Dictionary(train)
        
        # Term Document Frequency
        self.corpus = [self.id2word.doc2bow(text) for text in train]
        
        # Result dictionary
        self.model_results = {'Topics': [],
                         'Alpha': [],
                         'Beta': [],
                         'Coherence': []
                        }

        # Can take a long time to run
#         print('Validating hyperparameters...')
        if 1 == 1:
            pbar = tqdm.tqdm(total=(len(beta)*(len(alpha))*len(topics_range)), position=0, leave=True)
            # iterate through number of topics
            for k in topics_range:
                # iterate through alpha values
                for a in alpha:
                    # iterare through beta values
                    for b in beta:
                        # get the coherence score for the given parameters
                        cv = self.compute_coherence_values(corpus=self.corpus, id2word=self.id2word, text=train,
                                                      k=k, a=a, b=b)
                        # Save the model results
                        self.model_results['Topics'].append(k)
                        self.model_results['Alpha'].append(a)
                        self.model_results['Beta'].append(b)
                        self.model_results['Coherence'].append(cv)
#                         print('nb topics: {}, alpha: {}, beta: {}, coherence: {}'.format(k,a,b,cv))
                        pbar.update(1)
        self.cv_results = pd.DataFrame(self.model_results).sort_values("Coherence", ascending=False)
        self.cv_results.to_csv(self.name+'_lda_tuning_results.csv', index=False)
        display(self.cv_results)
        pbar.close()
        
        self.best_param = self.cv_results.iloc[0]
        self.best_num_topics = int(self.best_param['Topics'])
        self.best_alpha = self.best_param['Alpha']
        self.best_beta = self.best_param['Beta']
        name = self.name
        
        # Build LDA model
        print('Training best model...')
        self.lda_model = gensim.models.LdaMulticore(corpus=self.corpus,
                                                   id2word=self.id2word,
                                                   num_topics=self.best_num_topics,
                                                   alpha = self.best_alpha,
                                                   eta = self.best_beta,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
        print('Done.')
        pyLDAvis.enable_notebook()
        LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)
        pyLDAvis.save_html(LDAvis_prepared, name+'_'+str(self.best_alpha)+'_'+str(self.best_beta)+'_'+str(self.best_num_topics)+'.html')
        self.lda_model.save(name+'_'+str(self.best_alpha)+'_'+str(self.best_beta)+'_'+str(self.best_num_topics))
        pickle.dump(self.id2word, open( "id2word_"+name+'_'+str(self.best_alpha)+'_'+str(self.best_beta)+'_'+str(self.best_num_topics)+".pickle", "wb" ))
#         return self.lda_model, self.id2word
        
    def get_docs_topics(self):
        # Term Document Frequency
        corpus = [self.id2word.doc2bow(text) for text in self.docs]
        self.output = pd.concat([self.df, pd.DataFrame(gensim.matutils.corpus2csc(self.lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,self.best_num_topics+1)])], axis=1)
        return self.output

    def view(self):
        pyLDAvis.enable_notebook()
        self.LDAvis_prepared = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
        display(LDAvis_prepared)

In [23]:
all_lda = TopicModeller(reviews, 'tokens')
lda, id2word = all_lda.train_valid_lda()

  0%|                                                                                           | 0/32 [00:00<?, ?it/s]

Validating hyperparameters...


  3%|██▌                                                                             | 1/32 [03:21<1:44:19, 201.92s/it]

nb topics: 3, alpha: 0.1, beta: 0.1, coherence: 0.43450483502760867


  6%|█████                                                                           | 2/32 [06:59<1:43:18, 206.61s/it]

...

nb topics: 9, alpha: asymmetric, beta: 0.1, coherence: 0.5376250455256808


100%|███████████████████████████████████████████████████████████████████████████████| 32/32 [1:55:26<00:00, 216.45s/it]

nb topics: 9, alpha: asymmetric, beta: symmetric, coherence: 0.5188056235047867
Training best model...


Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.066201  0.118931       1        1  20.215582
0      0.136360  0.051055       2        1  18.425123
3      0.024137  0.137291       3        1  15.321939
2      0.140470  0.131328       4        1  12.971735
7     -0.088396  0.034800       5        1   8.255013
6      0.179293 -0.157810       6        1   6.515980
5     -0.007944 -0.353759       7        1   6.341194
8      0.066495  0.049640       8        1   6.129386
4     -0.384214 -0.011476       9        1   5.824048, topic_info=         Term          Freq         Total Category  logprob  loglift
6        love  47765.000000  47765.000000  Default  30.0000  30.0000
124  coverage  14228.000000  14228.000000  Default  29.0000  29.0000
25      shade   9271.000000   9271.000000  Default  28.0000  28.0000
43       long  15115.000000  15115.000000  Default  27.0000  27.0000
90       oily   9010.000000   9010.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
46       time   1088.588501  10269.834961   Topic9  -4.3967   0.5988
293       see    738.170349   3622.906250   Topic9  -4.7851   1.2523
80        new    733.587158   4315.590820   Topic9  -4.7913   1.0711
102      come    700.762451   5340.246094   Topic9  -4.8371   0.8123
88      would    653.378662  11844.045898   Topic9  -4.9071  -0.0542

[574 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
583       3  0.998049    absolute
438       2  0.830248  absolutely
438       3  0.169579  absolutely
167       1  0.987745      absorb
167       5  0.011203      absorb
...     ...       ...         ...
1651      5  0.998379       write
48        3  0.998023        year
48        6  0.001925        year
314       7  0.998891      yellow
437       8  0.998549        zone

[965 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 4, 3, 8, 7, 6, 9, 5])

In [8]:
grid = {
    "makeup_face":{},
    "makeup_lips":{},
    "makeup_eyes":{},
    "makeup_face_color":{},
    "makeup_other":{},
    "skincare_all":{}
}

In [9]:
grid['makeup_face']['df'] = reviews[(reviews['major_category']=='Makeup') & (reviews['application']=='Face')]
grid['makeup_lips']['df'] = reviews[(reviews['major_category']=='Makeup') & (reviews['application']=='Lips')]
grid['makeup_eyes']['df'] = reviews[(reviews['major_category']=='Makeup') & (reviews['application']=='Eyes')]
grid['makeup_face_color']['df'] = reviews[(reviews['major_category']=='Makeup') & (reviews['application']=='Face Color')]
grid['makeup_other']['df'] = reviews[(reviews['major_category']=='Makeup') & (~reviews['application'].isin(['Face', 'Lips', 'Eyes', 'Face Color']))]
grid['skincare_all']['df'] = reviews[reviews['major_category']=='Skincare']

In [17]:
grid['makeup_face']['best_param'] = {'Alpha': 'asymmetric',
                                    'Beta': 0.1}
grid['makeup_lips']['best_param'] = {'Alpha': 'asymmetric',
                                    'Beta': 0.1}
grid['makeup_eyes']['best_param'] = {'Alpha': 'asymmetric',
                                    'Beta': 'symmetric'}
grid['makeup_face_color']['best_param'] = {'Alpha': 'asymmetric',
                                    'Beta': 0.1}
grid['makeup_other']['best_param'] = {'Alpha': 0.1,
                                    'Beta': 'symmetric'}
grid['skincare_all']['best_param'] = {'Alpha': 'asymmetric',
                                    'Beta': 0.1}

In [72]:
for cat in grid.keys():
    print('Processing '+cat)
    grid[cat]['LDA'] = TopicModeller(df=grid[cat]['df'], column='tokens', name=cat)
    grid[cat]['LDA'].train_valid_lda(nb_samples = 50000,
                        topics_range = [2,4,6],
                        alpha = [0.1, 'symmetric','asymmetric'],
                        beta = [0.1, 'symmetric'])

Processing makeup_face


100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [28:22<00:00, 103.52s/it]

,Topics,Alpha,Beta,Coherence
17,6,asymmetric,symmetric,0.508606
16,6,asymmetric,0.1,0.501510
14,6,symmetric,0.1,0.499363
15,6,symmetric,symmetric,0.483744
13,6,0.1,symmetric,0.483396
12,6,0.1,0.1,0.483270
10,4,asymmetric,0.1,0.471039
11,4,asymmetric,symmetric,0.468145
6,4,0.1,0.1,0.465047
9,4,symmetric,symmetric,0.459581


100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [28:22<00:00, 94.57s/it]


Training best model...
Done.


...

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [31:12<00:00, 106.73s/it]

,Topics,Alpha,Beta,Coherence
17,6,asymmetric,symmetric,0.592819
16,6,asymmetric,0.1,0.584195
10,4,asymmetric,0.1,0.556307
8,4,symmetric,0.1,0.544013
11,4,asymmetric,symmetric,0.540895
15,6,symmetric,symmetric,0.538780
14,6,symmetric,0.1,0.531931
9,4,symmetric,symmetric,0.530704
13,6,0.1,symmetric,0.528157
12,6,0.1,0.1,0.523789


100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [31:12<00:00, 104.00s/it]


Training best model...
Done.


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.123307  0.110811       1        1  24.050776
2      0.078321 -0.057702       2        1  17.932777
3     -0.052616 -0.098739       3        1  17.339178
1     -0.312991  0.035775       4        1  15.671720
5      0.072429  0.211534       5        1  13.393360
4      0.091550 -0.201678       6        1  11.612185, topic_info=       Term          Freq         Total Category  logprob  loglift
53     love  16821.000000  16821.000000  Default  30.0000  30.0000
47      use  22060.000000  22060.000000  Default  29.0000  29.0000
109   great  12179.000000  12179.000000  Default  28.0000  28.0000
126    year   5019.000000   5019.000000  Default  27.0000  27.0000
51     star   5175.000000   5175.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
3     apply    534.430725   3147.094971   Topic6  -4.9302   0.3801
30   really    621.063416   6400.181152   Topic6  -4.7800  -0.1795
163    good    704.294739  12462.741211   Topic6  -4.6543  -0.7202
76   little    540.372131   3722.479492   Topic6  -4.9192   0.2232
26     need    500.860352   2824.453613   Topic6  -4.9951   0.4234

[409 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1796      2  0.995833    absolute
155       1  0.718557  absolutely
155       2  0.269053  absolutely
155       4  0.011380  absolutely
155       6  0.001626  absolutely
...     ...       ...         ...
1236      5  0.997388      yellow
805       1  0.133345       young
805       2  0.865035       young
1117      3  0.994931        zone
1117      5  0.003658        zone

[729 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 6, 5])

In [16]:
pickle.load(open( "grid.pickle", "rb" ))

AttributeError: Can't get attribute 'TopicModeller' on <module '__main__'>

In [21]:
def train_display_save(reviews, nb_samples, num_topics, alpha, eta, name):
    tokens=reviews['tokens']
    nb_samples = min(nb_samples, len(tokens))
        
    # Training set
    train = tokens.sample(nb_samples, random_state=3).values.tolist()

    # Create Dictionary
    id2word = corpora.Dictionary(train)

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in train]
    start=time.time()
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                                   id2word=id2word,
                                                   num_topics=num_topics,
                                                   alpha =alpha ,
                                                   eta =eta ,
                                                   random_state=3,
                                                   chunksize=100,
                                                   passes=10,
                                                   per_word_topics=True)
#     pyLDAvis.enable_notebook()
#     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#     display(LDAvis_prepared)
#     pyLDAvis.save_html(LDAvis_prepared, name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+'.html')
    lda_model.save(name+'_'+str(alpha).replace('.','')+'_'+str(eta).replace('.','')+'_'+str(num_topics))
#     pickle.dump(id2word, open( "id2word_"+ name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+".p", "wb" ))
    print(time.time()-start)
    

#     # Term Document Frequency
#     corpus = [id2word.doc2bow(text) for text in tokens]

#     topics = gensim.matutils.corpus2csc(lda.get_document_topics(corpus)).T.toarray()

#     topics = pd.DataFrame(topics, columns=['topic_'+str(i) for i in range(1,topics.shape[1]+1)])

#     output = pd.concat([reviews.reset_index(drop=True), topics], axis=1)
#     output.to_csv(name+'_'+str(alpha)+'_'+str(eta)+'_'+str(num_topics)+'.csv', index=False)
    return lda_model #, id2word, output, topics, reviews

In [ ]:
grid['']

In [ ]:
for cat in grid.keys():
    print(cat)
    num_topics = 4
    best_param = grid[cat]['best_param']
    grid[cat]['model'] = train_display_save(grid[cat]['df'], 10000, num_topics, best_param['Alpha'], best_param['Beta'], cat)

makeup_face
26.348291158676147
makeup_lips
19.98966121673584
makeup_eyes
23.80621910095215
makeup_face_color
25.18797731399536
makeup_other


In [138]:
pickle.dump(grid, open( "grid_4_topics.pickle", "wb" ))

In [131]:
pd.concat([rev.reset_index(drop=True), topics], axis=1)

,type,channel,source_product_identifier,date,onlinepost_id,description,nb_statements,rating,rating_1,rating_2,rating_3,rating_4,rating_5,sentiment_negative,sentiment_neutral,sentiment_positive,sentiment,tokens,product,brand_abbrev,elc_brand,brand_score,item_description,product_score,itemid_4,major_category_id,major_category,application_id,application,category_id,category,sub_category_id,sub_category,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
0,Skincare,Amazon USA,B0006LNFXW,2015-02,OnlinePost_20191120_140735168,I'm glad I found a container here on Amazon.. ...,2,5.0,0,0,0,0,1,0.0,0.00,1.00,1.00,"[find, container, look, kiss, sparkly]",MAC Iridescent Powder/Loose GOLDEN BRONZE,MC,M.A.C,1.0,IRIDESCENT POWDER/LOOSE,0.879657,SF96,510.0,Makeup,25,Face,26,Face Powder,44,Loose Face Powder,0.243224,0.486226,0.021969,0.018321,0.015684,0.013724,0.012199,0.178673,0.000000
1,Skincare,Amazon USA,B0006LNFXW,2015-02,OnlinePost_20200301_28485b30-1b33-4c9a-b2b6-c3...,it's really pretty and shimmery and is also gr...,4,5.0,0,0,0,0,1,0.0,0.00,1.00,1.00,"[really, pretty, shimmery, also, great, go, lo...",MAC Iridescent Powder/Loose GOLDEN BRONZE,MC,M.A.C,1.0,IRIDESCENT POWDER/LOOSE,0.879657,SF96,510.0,Makeup,25,Face,26,Face Powder,44,Loose Face Powder,0.349953,0.013764,0.010980,0.233745,0.000000,0.162322,0.000000,0.210323,0.000000
2,Skincare,Amazon USA,B0007D02HQ,2015-02,OnlinePost_20191120_142084340,Two Stars.,1,2.0,0,1,0,0,0,0.0,0.00,1.00,1.00,[star],"Clinique Continuous Coverage Makeup SPF 15, 1 ...",CL,Clinique,1.0,CONTINUOUS COVERAGE SPF15,0.716561,647G,510.0,Makeup,25,Face,28,Foundation,26,Liquid Foundation,0.109661,0.082246,0.065797,0.054831,0.546983,0.041123,0.036554,0.032898,0.029908
3,Skincare,Amazon USA,B0007D02HQ,2015-02,OnlinePost_20200301_71e4c00b-73f4-4931-a016-04...,didn't like the coverage.,1,2.0,0,1,0,0,0,0.0,0.00,1.00,1.00,[coverage],"Clinique Continuous Coverage Makeup SPF 15, 1 ...",CL,Clinique,1.0,CONTINUOUS COVERAGE SPF15,0.716561,647G,510.0,Makeup,25,Face,28,Foundation,26,Liquid Foundation,0.110480,0.082702,0.065825,0.054866,0.047000,0.041124,0.535194,0.032899,0.029908
4,Skincare,Amazon USA,B0007D02HQ,2015-03,OnlinePost_20191120_142437265,"I have used this product, and was very satisfi...",4,4.0,0,0,0,1,0,0.0,0.50,0.50,0.50,"[use, satisfied, satisfied, however, big, rip]","Clinique Continuous Coverage Makeup SPF 15, 1 ...",CL,Clinique,1.0,CONTINUOUS COVERAGE SPF15,0.716561,647G,510.0,Makeup,25,Face,28,Foundation,26,Liquid Foundation,0.504181,0.407705,0.018847,0.015674,0.013434,0.011755,0.010449,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134703,Cosmetics,Ulta,xlsImpprod4930101,2018-03,OnlinePost_20191027_185345917,My favorite primer.. Be sure to let lotion/ser...,4,5.0,0,0,0,0,1,0.0,0.25,0.75,0.75,"[favorite, sure, let, thoroughly, applying, be...",Photo Finish Protect SPF 20 Primer,SX,Smashbox,1.0,PHOTO FINISH PRIMER SPF 20 DS,0.872896,C0PL,510.0,Makeup,25,Face,29,Makeup Primers/Finis,31,AllOther Primers/Fin,0.000000,0.439583,0.244418,0.092170,0.066475,0.100740,0.000000,0.000000,0.043911
134704,Cosmetics,Ulta,xlsImpprod4930101,2018-06,OnlinePost_20191027_184229144,I just love love love this…. Yes I love it…. I...,4,5.0,0,0,0,0,1,0.0,0.00,1.00,1.00,"[love, love, love, love, wish, high, wish, high]",Photo Finish Protect SPF 20 Primer,SX,Smashbox,1.0,PHOTO FINISH PRIMER SPF 20 DS,0.872896,C0PL,510.0,Makeup,25,Face,29,Makeup Primers/Finis,31,AllOther Primers/Fin,0.025196,0.018297,0.014643,0.012187,0.010445,0.000000,0.000000,0.895324,0.000000
134705,Cosmetics,Ulta,xlsImpprod4930101,2018-06,OnlinePost_20191027_185282872,I love this product!. I love the way it makes ...,4,5.0,0,0,0,0,1,0.0,0.00,1.00,1.00,"[love, love, way, make, silky, look, great, go...",Photo Finish Protect SPF 20 Primer,SX,Smashbox,1.0,PHOTO FINISH PRIMER SPF 20 DS,0.872896,C0PL,510.0,Ma

In [25]:
lda, id2word = train_display_save(reviews['tokens'], 500000, 8, alpha='asymmetric', eta='symmetric', name='full')

In [26]:
docs = reviews['tokens'].values.tolist()

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

topics = gensim.matutils.corpus2csc(lda.get_document_topics(corpus)).T.toarray()

topics = pd.DataFrame(topics, columns=['topic_'+str(i) for i in range(1,topics.shape[1]+1)])

output = pd.concat([reviews, topics], axis=1)

output.to_pickle('reviews_w_topics.pkl')

In [80]:
topics = lda.print_topics(num_words=6)
for topic in topics:
    print('Topic #{}'.format(topic[0]+1))
    print('Terms: ', topic[1])

Topic #1
Terms:  0.033*"buy" + 0.031*"product" + 0.022*"so" + 0.022*"will" + 0.021*"use" + 0.020*"try"
Topic #2
Terms:  0.041*"love" + 0.041*"sample" + 0.034*"receive" + 0.033*"size" + 0.029*"all" + 0.027*"shadow"
Topic #3
Terms:  0.056*"skin" + 0.036*"primer" + 0.028*"product" + 0.027*"make" + 0.026*"feel" + 0.026*"use"
Topic #4
Terms:  0.038*"shade" + 0.034*"color" + 0.029*"too" + 0.025*"skin" + 0.025*"light" + 0.017*"more"
Topic #5
Terms:  0.069*"use" + 0.049*"good" + 0.048*"love" + 0.047*"year" + 0.044*"product" + 0.034*"ever"
Topic #6
Terms:  0.071*"product" + 0.054*"great" + 0.045*"eye" + 0.035*"star" + 0.034*"use" + 0.031*"concealer"
Topic #7
Terms:  0.064*"foundation" + 0.037*"coverage" + 0.029*"skin" + 0.028*"look" + 0.025*"day" + 0.020*"powder"
Topic #8
Terms:  0.066*"mascara" + 0.038*"lash" + 0.035*"eye" + 0.021*"eyeliner" + 0.021*"look" + 0.019*"love"
Topic #9
Terms:  0.097*"color" + 0.051*"lip" + 0.048*"love" + 0.036*"lipstick" + 0.022*"great" + 0.020*"look"


In [41]:
l15_ train_display_save(reviews['tokens'], 100000, 15, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.108425 -0.079646       1        1  13.448458
0     -0.159328  0.022728       2        1  11.095933
1     -0.104996  0.081584       3        1  10.091497
3     -0.098783 -0.037815       4        1   9.368036
4     -0.014167 -0.228340       5        1   8.402237
5      0.077708 -0.199683       6        1   7.674632
12    -0.169285  0.219647       7        1   7.091044
7     -0.116372 -0.068847       8        1   5.979372
9     -0.175805  0.153616       9        1   5.654981
8     -0.045745 -0.036242      10        1   5.004925
13     0.155779  0.119923      11        1   4.245401
10     0.114483 -0.109813      12        1   3.787817
14     0.159328 -0.187089      13        1   3.585887
11     0.190795  0.123635      14        1   2.774604
6      0.294811  0.226340      15        1   1.795181, topic_info=               Term          Freq         Total Category  logprob  loglift
26             love  48591.000000  48591.000000  Default  30.0000  30.0000
28         coverage  14049.000000  14049.000000  Default  29.0000  29.0000
22              use  42711.000000  42711.000000  Default  28.0000  28.0000
1               day  19339.000000  19339.000000  Default  27.0000  27.0000
37             good  28664.000000  28664.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
425           money    491.950806   2005.094971  Topic15  -4.0139   2.6150
1077  disappointing    204.412033    384.844421  Topic15  -4.8922   3.3874
913            pass    133.396942    224.067917  Topic15  -5.3190   3.5014
51             well    290.400635  16878.169922  Topic15  -4.5411  -0.0425
58            great    284.750153  29494.097656  Topic15  -4.5607  -0.6203

[889 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
334       2  0.999810  absolutely
804       1  0.997767      absorb
158       3  0.321797         add
158       4  0.678071         add
1122     11  0.991691  additional
...     ...       ...         ...
700       5  0.852478       wrong
700      11  0.146570       wrong
24        8  0.999896        year
779      15  0.999085      yellow
1446     13  0.996202    youthful

[1502 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2, 4, 5, 6, 13, 8, 10, 9, 14, 11, 15, 12, 7])

365.79632544517517


(<gensim.models.ldamulticore.LdaMulticore at 0x17d600c82c8>,
 <gensim.corpora.dictionary.Dictionary at 0x17cc8b54ec8>)

In [37]:
l10 = train_display_save(reviews['tokens'], 100000, 10, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.107283  0.005949       1        1  21.203896
2     -0.192958 -0.017620       2        1  16.234818
1     -0.064487  0.135172       3        1  13.132318
5     -0.158793 -0.146120       4        1  11.747620
9     -0.050967  0.130989       5        1  11.489159
3     -0.127931  0.110716       6        1   9.914849
8     -0.057595 -0.098704       7        1   5.349467
7      0.203712  0.019621       8        1   4.562047
4      0.218050 -0.343312       9        1   3.868961
6      0.338251  0.203310      10        1   2.496861, topic_info=          Term          Freq         Total Category  logprob  loglift
22         use  41656.000000  41656.000000  Default  30.0000  30.0000
26        love  47155.000000  47155.000000  Default  29.0000  29.0000
58       great  28803.000000  28803.000000  Default  28.0000  28.0000
12          go  20696.000000  20696.000000  Default  27.0000  27.0000
28    coverage  13976.000000  13976.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
883      photo    290.109680    904.904602  Topic10  -4.8719   2.5526
181      thank    369.401337   1770.703491  Topic10  -4.6303   2.1229
328  packaging    313.832977   1254.639893  Topic10  -4.7933   2.3044
148      order    327.512329   2711.295898  Topic10  -4.7507   1.5765
326     sticky    261.236633   1543.594971  Topic10  -4.9768   1.9137

[604 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
157       9  0.998210    absolute
334       1  0.846479  absolutely
334       3  0.153458  absolutely
804       2  0.966890      absorb
804       8  0.030905      absorb
...     ...       ...         ...
779       3  0.999320      yellow
400       3  0.997884       young
1446      6  0.994628    youthful
869       3  0.266476        zone
869       6  0.732511        zone

[954 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 6, 10, 4, 9, 8, 5, 7])

274.0791862010956


(<gensim.models.ldamulticore.LdaMulticore at 0x17cda4a5f88>,
 <gensim.corpora.dictionary.Dictionary at 0x17cda4c3788>)

In [38]:
l_9 = train_display_save(reviews['tokens'], 100000, 9, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.151741  0.051543       1        1  24.790617
2     -0.161115  0.014512       2        1  18.228716
1     -0.038584  0.107341       3        1  13.811959
5     -0.091467 -0.135222       4        1  12.226207
3     -0.111358  0.008439       5        1  10.295245
4     -0.091283 -0.117037       6        1   8.093184
8     -0.094714  0.052877       7        1   8.058111
7      0.353673  0.291231       8        1   2.454790
6      0.386590 -0.273684       9        1   2.041179, topic_info=            Term          Freq         Total Category  logprob  loglift
26          love  46463.000000  46463.000000  Default  30.0000  30.0000
22           use  41437.000000  41437.000000  Default  29.0000  29.0000
28      coverage  14325.000000  14325.000000  Default  28.0000  28.0000
24          year   8837.000000   8837.000000  Default  27.0000  27.0000
136          try  16338.000000  16338.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
558   everywhere    269.289307    353.247650   Topic9  -4.7449   3.6203
214       expect    465.747620   1781.423584   Topic9  -4.1971   2.5501
425        money    439.550659   1920.520630   Topic9  -4.2550   2.4170
42         cover    245.161606   5388.434570   Topic9  -4.8388   0.8016
1441     correct    179.028717    493.465424   Topic9  -5.1532   2.8777

[539 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
334       1  0.995286  absolutely
334       3  0.004261  absolutely
2625      7  0.195415      actual
2625      8  0.804315      actual
158       3  0.347665         add
...     ...       ...         ...
24        1  0.060421        year
24        3  0.004413        year
24        6  0.935061        year
779       3  0.998015      yellow
400       3  0.997969       young

[875 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 6, 4, 5, 9, 8, 7])

277.3664469718933


(<gensim.models.ldamulticore.LdaMulticore at 0x17cf0dbd408>,
 <gensim.corpora.dictionary.Dictionary at 0x17cf0dbd0c8>)

In [39]:
train_display_save(reviews['tokens'], 100000, 8, alpha='asymmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.136691  0.008281       1        1  29.412922
2     -0.158328 -0.003194       2        1  19.513187
5     -0.064077  0.008908       3        1  15.194673
1     -0.076407 -0.085318       4        1  12.682242
3     -0.140450 -0.120994       5        1   9.347183
4     -0.071685  0.043858       6        1   7.677266
7      0.210483  0.370894       7        1   4.234589
6      0.437155 -0.222435       8        1   1.937933, topic_info=          Term          Freq         Total Category  logprob  loglift
22         use  41230.000000  41230.000000  Default  30.0000  30.0000
24        year   8933.000000   8933.000000  Default  29.0000  29.0000
26        love  46079.000000  46079.000000  Default  28.0000  28.0000
28    coverage  14144.000000  14144.000000  Default  27.0000  27.0000
18        nice   8644.000000   8644.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
197       move    254.864853    549.323059   Topic8  -4.7480   3.1756
735   cosmetic    282.673553    727.832642   Topic8  -4.6445   2.9978
1089      wand    251.371262    832.594849   Topic8  -4.7618   2.7459
77        real    246.599152    880.685913   Topic8  -4.7810   2.6706
148      order    179.036163   2682.895752   Topic8  -5.1012   1.2365

[504 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
334       1  0.951752  absolutely
334       4  0.048051  absolutely
5261      8  0.989088    accurate
2625      3  0.998202      actual
158       4  0.200744         add
...     ...       ...         ...
24        7  0.997703        year
779       4  0.999204      yellow
400       2  0.001967       young
400       4  0.997469       young
869       4  0.998882        zone

[833 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 6, 2, 4, 5, 8, 7])

294.06596064567566


(<gensim.models.ldamulticore.LdaMulticore at 0x17c2ac09788>,
 <gensim.corpora.dictionary.Dictionary at 0x17a7e2319c8>)

In [31]:
train_display_save(reviews['tokens'], 100000, 7, alpha='symmetric', eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.118104 -0.010693       1        1  20.290110
2      0.014560 -0.145870       2        1  19.437847
1     -0.065970 -0.097884       3        1  19.360294
3     -0.065640 -0.201709       4        1  13.120198
0      0.223173  0.040887       5        1  10.240201
4      0.124811  0.252209       6        1   9.819157
6     -0.349038  0.163060       7        1   7.732195, topic_info=          Term          Freq         Total Category  logprob  loglift
26        love  45834.000000  45834.000000  Default  30.0000  30.0000
37        good  27886.000000  27886.000000  Default  29.0000  29.0000
218      shade   9027.000000   9027.000000  Default  28.0000  28.0000
100  recommend   7596.000000   7596.000000  Default  27.0000  27.0000
28    coverage  14184.000000  14184.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
15        look   2018.469604  30175.013672   Topic7  -4.0624  -0.1449
159      blend   1153.179688   4936.708008   Topic7  -4.6223   1.1056
18        nice   1302.536133   8656.076172   Topic7  -4.5005   0.6658
529      matte    917.440491   2909.153809   Topic7  -4.8510   1.4057
19    purchase    916.242859   7559.958496   Topic7  -4.8523   0.4494

[470 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
157       3  0.999185    absolute
334       1  0.064053  absolutely
334       3  0.808745  absolutely
334       5  0.125798  absolutely
334       7  0.001443  absolutely
...     ...       ...         ...
879       2  0.998159     wrinkle
546       1  0.998304       write
24        3  0.372624        year
24        6  0.627320        year
779       7  0.999036      yellow

[763 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 2, 4, 1, 5, 7])

254.2011616230011


(<gensim.models.ldamulticore.LdaMulticore at 0x17c2abf5a08>,
 <gensim.corpora.dictionary.Dictionary at 0x17d460b8948>)

In [33]:
train_display_save(reviews['tokens'], 100000, 6, alpha=0.1, eta='symmetric', name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.008682  0.074738       1        1  27.201521
5     -0.069848  0.062646       2        1  24.165337
0     -0.216498  0.133120       3        1  15.254469
4     -0.139189 -0.250528       4        1  11.507354
1      0.201068 -0.135007       5        1  11.146103
3      0.215785  0.115031       6        1  10.725211, topic_info=        Term          Freq         Total Category  logprob  loglift
22       use  41098.000000  41098.000000  Default  30.0000  30.0000
26      love  46163.000000  46163.000000  Default  29.0000  29.0000
37      good  27927.000000  27927.000000  Default  28.0000  28.0000
24      year   8785.000000   8785.000000  Default  27.0000  27.0000
58     great  28312.000000  28312.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
57      feel   1698.203491  14424.145508   Topic6  -4.5624   0.0932
19  purchase   1344.338135   7492.757812   Topic6  -4.7960   0.5145
56  favorite   1279.860474   6486.702148   Topic6  -4.8452   0.6096
37      good   1413.948730  27927.642578   Topic6  -4.7456  -0.7507
51      well   1182.466797  16132.198242   Topic6  -4.9243  -0.3806

[419 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
157       4  0.825131    absolute
157       5  0.173397    absolute
334       1  0.357538  absolutely
334       2  0.164024  absolutely
334       3  0.006460  absolutely
...     ...       ...         ...
155       5  0.001200       would
155       6  0.000171       would
24        4  0.999804        year
24        5  0.000114        year
779       5  0.998900      yellow

[683 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 1, 5, 2, 4])

257.9453058242798


(<gensim.models.ldamulticore.LdaMulticore at 0x17cda4cf5c8>,
 <gensim.corpora.dictionary.Dictionary at 0x17cda4cfac8>)

In [34]:
train_display_save(reviews['tokens'], 100000, 5, alpha='asymmetric', eta=0.1, name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.088285 -0.123255       1        1  38.881653
2      0.057741 -0.025228       2        1  21.310360
1      0.266972  0.101711       3        1  16.076698
4     -0.194290 -0.187831       4        1  12.578615
3     -0.218707  0.234604       5        1  11.152671, topic_info=           Term          Freq         Total Category  logprob  loglift
26         love  47239.000000  47239.000000  Default  30.0000  30.0000
4       perfect  12408.000000  12408.000000  Default  29.0000  29.0000
28     coverage  14489.000000  14489.000000  Default  28.0000  28.0000
22          use  41719.000000  41719.000000  Default  27.0000  27.0000
218       shade   9124.000000   9124.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
51         well   2033.424927  16416.791016   Topic5  -4.4214   0.1049
241        give   1684.963867   9576.279297   Topic5  -4.6093   0.4559
277         bit   1379.537598   4045.522949   Topic5  -4.8093   1.1176
119  definitely   1489.262207   6461.470215   Topic5  -4.7328   0.7259
73      formula   1195.910889   3658.064941   Topic5  -4.9522   1.0755

[329 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
334       1  0.761426  absolutely
334       3  0.238390  absolutely
573       4  0.999381         ago
81        1  0.351162        also
81        2  0.384142        also
...     ...       ...         ...
24        2  0.070802        year
24        3  0.006365        year
24        4  0.422018        year
779       3  0.998761      yellow
869       2  0.999658        zone

[498 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])

239.18256402015686


(<gensim.models.ldamulticore.LdaMulticore at 0x17a7a4f3ac8>,
 <gensim.corpora.dictionary.Dictionary at 0x17c2abf9548>)

In [35]:
train_display_save(reviews['tokens'], 100000, 4, alpha='asymmetric', eta=0.1, name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.048937  0.113301       1        1  45.656040
2      0.016255  0.192460       2        1  21.628445
1      0.227958 -0.188650       3        1  19.286968
3     -0.293151 -0.117111       4        1  13.428550, topic_info=         Term          Freq         Total Category  logprob  loglift
28   coverage  14658.000000  14658.000000  Default  30.0000  30.0000
26       love  47292.000000  47292.000000  Default  29.0000  29.0000
4     perfect  12297.000000  12297.000000  Default  28.0000  28.0000
218     shade   9226.000000   9226.000000  Default  27.0000  27.0000
24       year   9139.000000   9139.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
39        say   1650.246948   4597.279297   Topic4  -4.8159   0.9832
225   however   1420.717407   3174.906494   Topic4  -4.9656   1.2037
18       nice   1782.349243   8849.050781   Topic4  -4.7389   0.4054
241      give   1667.700439   9581.803711   Topic4  -4.8053   0.2594
51       well   1457.063965  16495.017578   Topic4  -4.9404  -0.4188

[267 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
81        1  0.438240     also
81        2  0.384363     also
81        3  0.138432     also
81        4  0.038918     also
82        1  0.764610   always
...     ...       ...      ...
879       2  0.999780  wrinkle
24        1  0.000547     year
24        3  0.999502     year
779       3  0.999932   yellow
869       2  0.999463     zone

[375 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])

243.96960830688477


(<gensim.models.ldamulticore.LdaMulticore at 0x17a7a502908>,
 <gensim.corpora.dictionary.Dictionary at 0x17a7a502948>)

In [36]:
train_display_save(reviews['tokens'], 100000, 3, alpha='asymmetric', eta=0.1, name='full')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.051197 -0.172327       1        1  56.318577
1     -0.234401  0.054473       2        1  22.935804
2      0.183204  0.117854       3        1  20.745625, topic_info=          Term          Freq         Total Category  logprob  loglift
28    coverage  14852.000000  14852.000000  Default  30.0000  30.0000
4      perfect  12241.000000  12241.000000  Default  29.0000  29.0000
26        love  47664.000000  47664.000000  Default  28.0000  28.0000
126       oily   9208.000000   9208.000000  Default  27.0000  27.0000
218      shade   9150.000000   9150.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
100  recommend   2119.777344   7715.424805   Topic3  -5.0004   0.2809
285       keep   1958.638184   5117.345215   Topic3  -5.0795   0.6124
224       find   2029.788452   8679.255859   Topic3  -5.0438   0.1198
109       hour   1909.858276   4188.477051   Topic3  -5.1047   0.7875
12          go   1963.974854  20841.091797   Topic3  -5.0768  -0.7891

[211 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
81        1  0.465637     also
81        2  0.161196     also
81        3  0.373238     also
82        1  0.591062   always
82        2  0.408814   always
...     ...       ...      ...
155       2  0.002221    would
155       3  0.150506    would
879       3  0.999840  wrinkle
24        2  0.999993     year
869       3  1.000035     zone

[263 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

232.1398787498474


(<gensim.models.ldamulticore.LdaMulticore at 0x17cda4c6d48>,
 <gensim.corpora.dictionary.Dictionary at 0x17cda4c6708>)

In [40]:
train_display_save(reviews['tokens'], 100000, 2, alpha='asymmetric', eta=0.1, name='full')

PreparedData(topic_coordinates=              x    y  topics  cluster      Freq
topic                                          
0      0.163045  0.0       1        1  68.11438
1     -0.163045  0.0       2        1  31.88562, topic_info=         Term          Freq         Total Category  logprob  loglift
28   coverage  14801.000000  14801.000000  Default  30.0000  30.0000
15       look  31567.000000  31567.000000  Default  29.0000  29.0000
4     perfect  12308.000000  12308.000000  Default  28.0000  28.0000
218     shade   9067.000000   9067.000000  Default  27.0000  27.0000
66    natural   7133.000000   7133.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
126      oily   3433.560791   9129.950195   Topic2  -4.9480   0.1651
37       good   4621.998047  28572.935547   Topic2  -4.6507  -0.6786
81       also   3203.921143   8055.221680   Topic2  -5.0172   0.2211
12         go   3782.213623  20768.574219   Topic2  -4.8513  -0.5601
76       make   2861.595947  21584.181641   Topic2  -5.1302  -0.8776

[158 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
81        1  0.602218   also
81        2  0.397754   also
63        1  0.792860  apply
63        2  0.207058  apply
115       2  0.999812   area
...     ...       ...    ...
114       1  0.999919  worth
155       1  0.941564  would
155       2  0.058475  would
24        1  0.577177   year
24        2  0.422743   year

[174 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

243.65680503845215


(<gensim.models.ldamulticore.LdaMulticore at 0x17cefb4bb08>,
 <gensim.corpora.dictionary.Dictionary at 0x17cd1260d48>)

In [ ]:
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

## Splitting by review

In [ ]:
bad_reviews['tokens'].sample(100000)

In [ ]:
good_reviews_6 = train_display_save(tokens=good_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='good_reviews')

In [ ]:
bad_reviews_6 = train_display_save(tokens=bad_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='bad_reviews')

In [ ]:
neutral_reviews_6 = train_display_save(tokens=neutral_reviews['tokens'], nb_samples=100000, num_topics=6, alpha='asymmetric', eta=0.1, name='neutral_reviews')

In [ ]:
def save_display(self, title):
    p = pyLDAvis.gensim.prepare(self.lda_model, self.corpus, self.id2word)
    pyLDAvis.save_html(p, title+'.html')

In [ ]:
save_display(good_lda, 'good_reviews_8_topics')

In [ ]:
save_display(bad_lda, 'bad_reviews_8_topics')

In [ ]:
save_display(neutral_lda, 'neutral_reviews_8_topics')

In [ ]:
good_reviews = reviews[reviews['rating']>=4]
neutral_reviews = reviews[(reviews['rating']>2) & (reviews['rating']<4)]
bad_reviews = reviews[reviews['rating']<=2]

In [ ]:
print(len(good_reviews))
good_lda = TopicModeller(good_reviews, 'tokens')
good_lda.train_valid_lda()

In [ ]:
display(good_lda.cv_results)

In [ ]:
good_lda.view()

In [ ]:
print(len(bad_reviews))
bad_lda = TopicModeller(bad_reviews, 'tokens')
bad_lda.train_valid_lda()
bad_lda.view()

In [ ]:
display(bad_lda.cv_results)

In [ ]:
bad_lda.view()

In [ ]:
print(len(neutral_reviews))
neutral_lda = TopicModeller(neutral_reviews, 'tokens')
neutral_lda.train_valid_lda()
neutral_lda.view()

In [ ]:
display(neutral_lda.cv_results)

In [ ]:
neutral_lda.view()

In [ ]:
good_reviews = good_lda.get_docs_topics()

In [ ]:
neutral_reviews = neutral_lda.get_docs_topics()

In [ ]:
bad_reviews = bad_lda.get_docs_topics()

In [ ]:
good_reviews.to_pickle('good_reviews.pickle')
bad_reviews.to_pickle('bad_reviews.pickle')
neutral_reviews.to_pickle('neutral_reviews.pickle')

In [ ]:
good_lda.cv_results

In [ ]:
good_reviews_3 = train_display_save(tokens=good_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta=0.1, name='good_reviews')
bad_reviews_3 = train_display_save(tokens=bad_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta=0.1, name='bad_reviews')
neutral_reviews_3 = train_display_save(tokens=neutral_reviews['tokens'], nb_samples=100000, num_topics=3, alpha='asymmetric', eta='symmetric', name='neutral_reviews')

In [ ]:
int_start=time.time()
# Inputs
tokens_good = random.sample(good_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_good = corpora.Dictionary(tokens_good)
# Term Document Frequency
corpus_good = [id2word_good.doc2bow(text) for text in tokens_good]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_good = gensim.models.LdaMulticore(corpus=corpus_good,
                                       id2word=id2word_good,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 0.1,
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_good, corpus_good, id2word_good)
LDAvis_prepared

In [ ]:
pyLDAvis.gensim.prepare(lda_model_good, corpus_good, id2word_good)

In [ ]:
bad_lda.cv_results

In [ ]:
int_start=time.time()
# Inputs
tokens_bad = random.sample(bad_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_bad = corpora.Dictionary(tokens_bad)
# Term Document Frequency
corpus_bad = [id2word_bad.doc2bow(text) for text in tokens_bad]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_bad = gensim.models.LdaMulticore(corpus=corpus_bad,
                                       id2word=id2word_bad,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 0.1,
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_bad, corpus_bad, id2word_bad)
LDAvis_prepared

In [ ]:
int_start=time.time()
# Inputs
tokens_neutral = random.sample(neutral_reviews['tokens'].values.tolist(),100000)
# Create Dictionary
id2word_neutral = corpora.Dictionary(tokens_neutral)
# Term Document Frequency
corpus_neutral = [id2word_neutral.doc2bow(text) for text in tokens_neutral]
print(time.time()-int_start)

# Build LDA model
int_start=time.time()
lda_model_neutral = gensim.models.LdaMulticore(corpus=corpus_neutral,
                                       id2word=id2word_neutral,
                                       num_topics=3,
                                       alpha = 'asymmetric',
                                       eta = 'symmetric',
                                       random_state=3,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_neutral, corpus_neutral, id2word_neutral)
LDAvis_prepared

## NLP Preprocessing

In [ ]:
docs = pickle.load(open("docs.pickle", "rb"))

d = corpora.Dictionary(docs)

freq = pd.DataFrame(d.dfs.values(), index=d.dfs.keys(), columns=['freq'])
freq.index.name = 'idx'
freq = freq.reset_index()
freq['token'] = freq['idx'].apply(lambda x:d[x])
freq = freq.sort_values(by='freq', ascending=False)

In [ ]:
freq[freq['freq']>500]

## Validating Model

In [ ]:
# Inputs
docs = random.sample(docs, 100000)

# Create Dictionary
id2word = corpora.Dictionary(docs)
pickle.dump(id2word, open( "id2word.p", "wb" ) )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in docs]

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, text, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}

nb_words = len(id2word)

# Topics range
min_topics = 6
max_topics = 10
step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
topics_range = [8]
# Alpha parameter
# Added in the loop

# Alpha
alpha = [
#     0.1, 
#     'symmetric',
#     'asymmetric'
]

# Beta parameter
beta = [
    0.1, 
#     200/nb_words
]
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.05), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#     gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
    self.corpus
]

corpus_title = [
#     '25% Corpus',
#     '50% Corpus',
#     '75% Corpus',
    '100% Corpus'
]

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*(len(alpha)+1)*len(topics_range)))
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            alpha.append(50/k)
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, text=docs,
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    pbar.update(1)
    res = pd.DataFrame(model_results)
    res = pd.DataFrame(model_results).sort_values("Coherence", ascending=False)
    res.to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [ ]:
res

## Training Best Model

In [ ]:
best_param = res.iloc[0]
num_topics = best_param['Topics']
alpha = best_param['Alpha']
eta = best_param['Beta']

In [ ]:
# Build LDA model
int_start=time.time()
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       alpha = alpha,
                                       eta = eta,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
print(time.time()-int_start)

In [ ]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

In [ ]:
lda_model.save('lda_test.model')

In [ ]:
print(time.time()-start)

## Predictions

In [ ]:
# df = pd.read_pickle('reviews_concat.pkl')

In [ ]:
# df = df.loc['2019']

In [ ]:
# print(len(df))

In [ ]:
# docs = list(tqdm.tqdm(preprocess(df.values.tolist(), stop_words), position=0, leave=True))

In [ ]:
# pickle.dump(docs, open( "docs.p", "wb" ) )

In [ ]:
# docs = pickle.load(open("docs.p", "rb"))
# id2word = pickle.load(open("id2word.p", "rb"))

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in docs]

# output = pd.concat([df.to_frame('description').reset_index(), pd.DataFrame(gensim.matutils.corpus2csc(lda_model.get_document_topics(corpus)).T.toarray(), columns=['topic_'+str(i) for i in range(1,num_topics+1)])], axis=1, ignore_index=True)

# output.to_pickle('reviews_w_topics_test.pkl')

## Appendix

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=150) # higher threshold fewer phrases.
# # trigram = gensim.models.Phrases(bigram[data_words], threshold=150)

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# # trigram_mod = gensim.models.phrases.Phraser(trigram)

# def remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """https://spacy.io/api/annotation"""
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

# # Remove Stop Words
# data_words = remove_stopwords(data_words)

# # Do lemmatization keeping only noun, adj, vb, adv
# data_words = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # Form Bigrams
# data_words = make_bigrams(data_words)

In [ ]:
# # Build LDA model
# lda_model = gensim.models.LdaMulticore(corpus=corpus,
#                                        id2word=id2word,
#                                        num_topics=8, 
#                                        random_state=100,
#                                        chunksize=100,
#                                        passes=10,
#                                        per_word_topics=True)

In [ ]:
# pprint(lda_model.print_topics())
# doc_lda = lda_model[corpus]

In [ ]:
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('Coherence Score: ', coherence_lda)